In [3]:
! pip install -U -q odfpy SPARQLWrapper bigjson ijson rdflib

In [20]:
import pandas as pd
import numpy as np
import rdflib
import urllib.parse
import time
import pickle
#import cPickle as pickle
import bz2
import json
import ijson
import bigjson #https://github.com/henu/bigjson.git
from SPARQLWrapper import SPARQLWrapper, JSON  #https://rdflib.dev/sparqlwrapper/

from functools import lru_cache as cache

### Reading FoodOn vocabulary

In [15]:
vocabularies = pd.read_excel("../data/vocabularies/FoodOn_concepts.ods", engine="odf")
vocabularies.rename(columns={'Food Concept':'FoodOn'}, inplace=True)
vocabularies['FoodOn'] = vocabularies['FoodOn'].apply(str.title)
vocabularies.sort_values('FoodOn', inplace=True)
vocabularies.reset_index(inplace=True, drop=True)
vocabularies.head(10)

,FoodOn
0,Agar
1,Alcohol
2,Ale
3,Alligator
4,Almond
5,Almond Butter
6,Almond Paste
7,Aloe Vera
8,Aluminum
9,Amaranth


### Building a function to query SPARQL endpoints

In [5]:
def query_sparql_endpoint(endpoint, query, term):
    term = urllib.parse.quote(term)
    sparql = SPARQLWrapper(endpoint)
    sparql.setQuery(query%term)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    time.sleep(1)
    return(results["results"]["bindings"])

### Querying Getty [AAT Vocabulary](http://www.getty.edu/research/tools/vocabularies/aat/)  

[downloads](https://www.getty.edu/research/tools/vocabularies/obtain/download.html)

In [6]:
vocabularies['Getty-AAT'] = np.empty((len(vocabularies), 0)).tolist()
endpoint = "http://vocab.getty.edu/sparql"  #http://vocab.getty.edu/queries#_Toc485115879

query = '''
    SELECT ?Subject ?Term ?Parents ?Descr ?ScopeNote ?Type (coalesce(?Type1,?Type2) as ?ExtraType) {
  ?Subject luc:term "%s"; a ?typ.
  ?typ rdfs:subClassOf gvp:Subject; rdfs:label ?Type.
  FILTER (?typ != gvp:Subject)
  OPTIONAL {?Subject gvp:placeTypePreferred [gvp:prefLabelGVP [xl:literalForm ?Type1]]}
  OPTIONAL {?Subject gvp:agentTypePreferred [gvp:prefLabelGVP [xl:literalForm ?Type2]]}
  OPTIONAL {?Subject gvp:prefLabelGVP [xl:literalForm ?Term]}
  OPTIONAL {?Subject gvp:parentStringAbbrev ?Parents}
  OPTIONAL {?Subject foaf:focus/gvp:biographyPreferred/schema:description ?Descr}
  OPTIONAL {?Subject skos:scopeNote [dct:language gvp_lang:en; rdf:value ?ScopeNote]}}
'''

#for idx in vocabularies.index:
#    term = vocabularies.loc[idx,'FoodOn']
#    results = query_sparql_endpoint(endpoint, query, term)
#    for result in results:
#        vocabularies.loc[idx, 'Getty-AAT'].append({result["Term"]["value"]:result['Subject']['value']})

#vocabularies.to_pickle('./temp.pkl')

In [16]:
vocabularies = pd.read_pickle("./temp.pkl")
vocabularies.reset_index(inplace=True, drop=True)

In [17]:
vocabularies.head()

,FoodOn,Getty-AAT
0,Agar,"[{'Agar, Syrt': 'http://vocab.getty.edu/tgn/76..."
1,Alcohol,[{'alcohol (general)': 'http://vocab.getty.edu...
2,Ale,[{'Ale Water': 'http://vocab.getty.edu/tgn/746...
3,Alligator,"[{'Alligator, The': 'http://vocab.getty.edu/tg..."
4,Almond,[{'Almond': 'http://vocab.getty.edu/tgn/112112...


### Querying [Iconclass](http://www.iconclass.nl/home)  
#### Using [iconclass database](http://iconclass.org/data/iconclass_20200529_skos_jsonld.ndjson.gz). See Also [LOD](http://www.iconclass.org/help/lod) and [ML Experiment](https://labs.brill.com/ictestset/)  

In [21]:
@cache(maxsize=None)
def search_iconclas(term):
    results = []
    with open('../data/vocabularies/iconclass_20200710_skos_jsonld.ndjson', 'rb') as f:
        for line in f:
            element = json.loads(line)
            try:
                for item in element['skos:prefLabel']:
                    if item['@language'] == 'en':
                        if term.lower() in item['@value'].lower():
                            results.append({item['@value']:element['@id']})
            except:
                #print("-", end='')
                pass;
    return(results)

In [22]:
t0 = time.time()

vocabularies['Iconclass'] = np.empty((len(vocabularies), 0)).tolist()

for idx in vocabularies.index:
    term = vocabularies.loc[idx,'FoodOn']
    result = search_iconclas(term)
    vocabularies.loc[idx, 'Iconclass'].append(result)
    print(f"{idx}+", end='');
            
print(time.time() - t0)
vocabularies.to_pickle('./temp2.pkl')

0+

KeyboardInterrupt: 

In [ ]:
#vocabularies = pd.read_pickle("./temp2.pkl")

#### Alternatively querying SPARQL endpoint

### Querying [DBPedia](http://dbpedia.org/sparql)

In [ ]:
with bz2.BZ2File('../data/vocabularies.pbz2', 'r') as f:
    vocabularies = pickle.load(f)

In [ ]:
vocabularies.head(20)

#### Finding the exact Matches

In [ ]:
vocabularies['Getty_exact'] = np.NaN
vocabularies['Iclass_exact'] = np.NaN
for idx, item in enumerate(vocabularies.index):
    for item in vocabularies.loc[idx, 'Getty-ATT']:
        if item == vocabularies.loc[idx, 'FoodOn']:
            vocabularies.loc[idx, 'Getty_exact'] = item
            break
    for item in vocabularies.loc[idx, 'Iconclass']:
        if item == vocabularies.loc[idx, 'FoodOn']:
            vocabularies.loc[idx, 'Iclass_exact'] = item
            break

In [ ]:
vocabularies.dropna(axis=0, subset=['Getty_exact'])

In [ ]:
vocabularies.dropna(axis=0, subset=['Iclass_exact'])

#### Percent of exact matches

In [ ]:
p1 = len(vocabularies.dropna(axis=0, subset=["Getty_exact"]))/len(vocabularies) * 100
print(f'AAT: {p1:.2f}')
p2 = len(vocabularies.dropna(axis=0, subset=["Iclass_exact"]))/len(vocabularies) * 100
print(f'Iconclass: {p2:.2f}')

In [ ]:
vocabularies.to_excel("../data/vocabularies.xlsx")

## TBD
### Querying Europeana

https://pro.europeana.eu/page/search  
https://pro.europeana.eu/page/sparql  
http://matthewlincoln.net/2014/07/10/sparql-for-humanists.html  